In [1]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import StrMethodFormatter

# Import our brand new custom package
import fintool

# Set a nice theme for the plots
sns.set_theme(style="whitegrid")

✅ Personal finance toolkit 'fintool' initialized.


In [2]:
import json
import pandas as pd

# Load master account list for Asset & Liability reports
try:
    master_accounts_raw = json.load(open('finance_data/accounts_master.json'))
    master_accounts_df = pd.DataFrame(master_accounts_raw)
    print("✅ Master accounts data loaded.")
except FileNotFoundError:
    print("❌ Error: 'finance_data/accounts_master.json' not found.")
    master_accounts_df = pd.DataFrame()

# Load budget items for the Budget report
try:
    budget_items_raw = json.load(open('finance_data/budget_items.json'))
    budget_items_df = pd.DataFrame(budget_items_raw)
    print("✅ Budget items data loaded.")
except FileNotFoundError:
    print("❌ Error: 'finance_data/budget_items.json' not found.")
    budget_items_df = pd.DataFrame()

# Load savings rules data
try:
    savings_data = json.load(open('finance_data/savings_rules.json'))
    print("✅ Savings rules data loaded.")
except FileNotFoundError:
    print("❌ Error: 'finance_data/savings_rules.json' not found.")
    savings_data = {}

# Load transactions for spending analysis
try:
    # fintool.load_data() returns three items; we only need the second one (transactions)
    _, past_transactions, _ = fintool.load_data()
    transactions_df = pd.DataFrame([t.model_dump() for t in past_transactions])
    print("✅ Transactional data loaded.")
except (FileNotFoundError, TypeError, ValueError):
    print("⚠️ Warning: Transactional data not loaded or file is empty.")
    transactions_df = pd.DataFrame()

✅ Master accounts data loaded.
✅ Budget items data loaded.
✅ Savings rules data loaded.
✅ Transactional data loaded.


In [3]:
# Generate the high-level cash flow summary
cashflow_summary_df = fintool.generate_cashflow_summary(
    master_accounts_df, 
    budget_items_df, 
    savings_data
)

# Display the formatted report
display(cashflow_summary_df.style.format({
    'Monthly': '${:,.2f}',
    'Half': '${:,.2f}'
}).set_caption("<h2>Monthly Cash Flow Summary</h2>"))

,Monthly,Half
Total Income,"$6,178.24","$3,089.12"
Total Expenses,"$3,137.23","$1,568.61"
Total Savings,"$1,009.64",$504.82
Remainder,"$2,031.37","$1,015.69"


In [4]:
# Generate the budget report
budget_report_df = fintool.generate_budget_report(budget_items_df)

# Display the formatted report
display(budget_report_df.rename(columns={
    'item_name': 'Bill/Budget Item',
    'due_day': 'Due Day',
    'monthly_cost': 'Monthly Cost',
    'per_paycheck_cost': 'Per Paycheck'
}).style.format({
    'Due Day': '{:.0f}',
    'Monthly Cost': '${:,.2f}',
    'Per Paycheck': '${:,.2f}'
}).set_caption("<h2>Monthly Bills & Budget</h2>").hide(axis='index'))

Bill/Budget Item,Due Day,Monthly Cost,Per Paycheck
Rent,1,"$1,600.00",$800.00
Google Cloud Platform,2,$4.00,$2.00
Utilities,2,$200.00,$100.00
Dog Food,6,$43.87,$21.93
Wells Fargo Account Fee,8,$10.00,$5.00
Dashlane,14,$7.03,$3.52
ATT - Cell Bill,15,$112.62,$56.31
Classpass,16,$79.00,$39.50
Apple Music,17,$18.39,$9.20
Lemonade Pet Insurance,24,$55.53,$27.77


In [5]:
# Generate the savings report
savings_report_df = fintool.generate_savings_report(savings_data)

# Display the formatted report
display(savings_report_df.rename(columns={
    'contribution_name': 'Savings Contribution',
    'monthly_amount': 'Monthly',
    'per_paycheck_contribution': 'Half'
}).style.format({
    'Monthly': '${:,.2f}',
    'Half': '${:,.2f}'
}).set_caption("<h2>Savings Contributions</h2>").hide(axis='index'))

KeyError: 'per_paycheck_cost'

In [ ]:
# Generate the Assets report
assets_report_df = fintool.generate_assets_report(master_accounts_df)

# Display the formatted report
display(assets_report_df.style.format({
    'value': '${:,.2f}',
    'monthly_income': '${:,.2f}',
    'apy': '{:.2%}'
}).set_caption("<h2>Assets Statement</h2>").hide(axis='index'))

account_name,value,monthly_income,apy,asset_class
Wealthfront Cash - Bills,$108.83,"$6,178.24",4.50%,Cash
Total Assets,$108.83,"$6,178.24",4.50%,nan


In [ ]:
# Generate the advanced liabilities report
liabilities_report_df = fintool.generate_liabilities_report(master_accounts_df)

# Display the formatted report
display(liabilities_report_df.style.format({
    'value': '${:,.2f}',
    'monthly_payment': '${:,.2f}',
    'current_apr': '{:.2%}',
    'credit_limit': '${:,.0f}',
    'available_credit': '${:,.0f}',
    'due_day': '{:.0f}'
}).set_caption("<h2>Liabilities Statement</h2>").hide(axis='index'))

account_name,value,monthly_payment,current_apr,due_day,status,credit_limit,available_credit,notes
My Best Buy Visa Credit Card,$642.78,$30.00,0.00%,7,None,"$4,000","$3,357","att, stolen phone"
American Express Blue Cash Preferred,"$3,889.86",$118.00,9.99%,17,Payment Plan,"$13,000","$9,110",None
Federal Student Loan 1,"$3,485.13",$36.90,3.76%,13,None,$nan,$nan,None
Earnest Private Student Loan,"$57,557.06",$464.79,5.73%,3,None,$nan,$nan,None
Total Liabilities,"$65,574.83",$649.69,5.82%,nan,nan,"$17,000","$12,467",nan


In [ ]:
if not transactions_df.empty:
    # Filter for expenses only from the last 30 days
    transactions_df['date'] = pd.to_datetime(transactions_df['date'])
    thirty_days_ago = pd.Timestamp.now() - pd.Timedelta(days=30)
    
    expenses_df = transactions_df[
        (transactions_df['amount'] < 0) & 
        (transactions_df['date'] >= thirty_days_ago)
    ].copy()
    
    if not expenses_df.empty:
        # Make amount positive for easier plotting
        expenses_df['amount'] = expenses_df['amount'].abs()
        
        spending_by_category = expenses_df.groupby('category')['amount'].sum().sort_values(ascending=False)
        
        # --- Create Bar Chart ---
        fig, ax = plt.subplots(figsize=(12, 7))
        spending_by_category.plot(kind='barh', ax=ax)
        
        ax.set_title('Spending by Category (Last 30 Days)', fontsize=16)
        ax.set_xlabel('Total Spent ($)', fontsize=12)
        ax.set_ylabel('Category', fontsize=12)
        
        # Format the x-axis to show dollar signs
        ax.xaxis.set_major_formatter(StrMethodFormatter('${x:,.0f}'))
        
        plt.tight_layout()
        plt.show()
    else:
        print("No expense transactions found in the last 30 days.")
else:
    print("Transactional data is empty. Cannot generate spending report.")

item_name,due_day,monthly_cost,per_paycheck_cost
Rent,1,"$1,600.00",$800.00
Utilities,2,$200.00,$100.00
Dog Food,6,$43.87,$21.93
Internet,29,$75.68,$37.84
Psychiatry,nan,$45.00,$22.50
Groceries,nan,$600.00,$300.00
Gas,nan,$35.00,$17.50
Total Bills,nan,"$2,599.55","$1,299.77"
